<h2> Featurizing text data with tfidf weighted word-vectors </h2>

In [13]:
import pandas as pd
import matplotlib.pyplot as plt
import re
import time
import warnings
import numpy as np
from nltk.corpus import stopwords
from sklearn.preprocessing import normalize
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfVectorizer
warnings.filterwarnings("ignore")
import spacy

In [15]:
df =pd.read_csv(r'D:\python\Quora Question Pair\Data_Set\train.csv')
df['question1'] = df['question1'].apply(lambda x: str(x))
df['question2'] = df['question2'].apply(lambda x: str(x))

id  qid1  qid2                                          question1  \
0   0     1     2  What is the step by step guide to invest in sh...   
1   1     3     4  What is the story of Kohinoor (Koh-i-Noor) Dia...   
2   2     5     6  How can I increase the speed of my internet co...   
3   3     7     8  Why am I mentally very lonely? How can I solve...   
4   4     9    10  Which one dissolve in water quikly sugar, salt...   

                                           question2  is_duplicate  
0  What is the step by step guide to invest in sh...             0  
1  What would happen if the Indian government sto...             0  
2  How can Internet speed be increased by hacking...             0  
3  Find the remainder when [math]23^{24}[/math] i...             0  
4            Which fish would survive in salt water?             0

In [17]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.feature_extraction.text import CountVectorizer

questions =list(df['question1'])+list(df['question2'])

tfidf = TfidfVectorizer(lowercase=False, )  # make a vectorizer
tfidf.fit_transform(questions)              # apply the vectorizer
# dict key:word and value:tf-idf score
word2tfidf = dict(zip(tfidf.get_feature_names(), tfidf.idf_))

- After we find TF-IDF scores, we convert each question to a weighted average of word2vec vectors by these scores.
- here we use a pre-trained GLOVE model which comes free with "Spacy". Very similar to Word2Vec.
- It is trained on Wikipedia and therefore, it is stronger in terms of word semantics. 

In [20]:
import en_core_web_sm
nlp = en_core_web_sm.load()
from tqdm import tqdm
vecs1 = []
# tqdm is used to print the progress bar
for qu1 in tqdm(list(df['question1'])):
    doc1 = nlp(qu1) 
    # 384 is the number of dimensions of vectors 
    mean_vec1 = np.zeros([len(doc1), len(doc1[0].vector)])
    for word1 in doc1:
        # word2vec
        vec1 = word1.vector
        # fetch df score
        try:
            idf = word2tfidf[str(word1)]
        except:
            idf = 0
        # compute final vec
        mean_vec1 += vec1 * idf
    mean_vec1 = mean_vec1.mean(axis=0)
    vecs1.append(mean_vec1)
df['q1_feats_m'] = list(vecs1)


100%|█████████████████████████████████████████████████████████████████████████| 404290/404290 [43:21<00:00, 155.44it/s]


In [31]:
vecs2 = []
for qu2 in tqdm(list(df['question2'])):
    doc2 = nlp(qu2) 
    mean_vec2 = np.zeros([len(doc2), len(doc2[0].vector)])
    for word2 in doc2:
        # word2vec
        vec2 = word2.vector
        # fetch df score
        try:
            idf = word2tfidf[str(word2)]
        except:
            #print word
            idf = 0
        # compute final vec
        mean_vec2 += vec2 * idf
    mean_vec2 = mean_vec2.mean(axis=0)
    vecs2.append(mean_vec2)
df['q2_feats_m'] = list(vecs2)

100%|█████████████████████████████████████████████████████████████████████████| 404290/404290 [48:02<00:00, 140.28it/s]


In [32]:
#prepro_features_train.csv (Simple Preprocessing Feartures)
#nlp_features_train.csv (NLP Features)
import os
if os.path.isfile('nlp_features_train.csv'):
    dfnlp = pd.read_csv("nlp_features_train.csv",encoding='latin-1')
if os.path.isfile('df_fe_without_preprocessing_train.csv'):
    dfppro = pd.read_csv("df_fe_without_preprocessing_train.csv",encoding='latin-1')

In [33]:
df1 = dfnlp.drop(['qid1','qid2','question1','question2'],axis=1)
df2 = dfppro.drop(['qid1','qid2','question1','question2','is_duplicate'],axis=1)
df3 = df.drop(['qid1','qid2','question1','question2','is_duplicate'],axis=1)
df3_q1 = pd.DataFrame(df3.q1_feats_m.values.tolist(), index= df3.index)
df3_q2 = pd.DataFrame(df3.q2_feats_m.values.tolist(), index= df3.index)

In [34]:
# dataframe of nlp features
df1.head()

id  is_duplicate  freq_quid1  freq_quid2  q1len  q2len  q1_n_words  \
0   0             0           1           1     66     57          14   
1   1             0           4           1     51     88           8   
2   2             0           1           1     73     59          14   
3   3             0           1           1     50     65          11   
4   4             0           3           1     76     39          13   

   q2_n_words  word_common  word_Total          ...            ctc_max  \
0          12         12.0        23.0          ...           0.785709   
1          13          8.0        20.0          ...           0.466664   
2          10         14.0        24.0          ...           0.285712   
3           9         10.0        19.0          ...           0.000000   
4           7         13.0        20.0          ...           0.307690   

   last_word_eq  first_word_eq  abs_len_diff  mean_len  token_set_ratio  \
0           0.0            1.0           2.0      13.0              100   
1           0.0            1.0           5.0      12.5               86   
2           0.0            1.0           4.0      12.0               63   
3           0.0            0.0           2.0      12.0               28   
4           0.0            1.0           6.0      10.0               67   

   token_sort_ratio  fuzz_ratio  fuzz_partial_ratio  longest_substr_ratio  
0                93          93                 100              0.982759  
1                63          66                  75              0.596154  
2                63          43                  47              0.166667  
3                24           9                  14              0.039216  
4                47          35                  56              0.175000  

[5 rows x 30 columns]

In [35]:
df2.head()

id  freq_quid1  freq_quid2  q1len  q2len  q1_n_words  q2_n_words  \
0   0           1           1     66     57          14          12   
1   1           4           1     51     88           8          13   
2   2           1           1     73     59          14          10   
3   3           1           1     50     65          11           9   
4   4           3           1     76     39          13           7   

   word_common  word_Total  word_share  freq_qid1  freq_qid2  freq_q1+q2  \
0         12.0        23.0    0.434783          1          1           2   
1          8.0        20.0    0.200000          4          1           5   
2         14.0        24.0    0.166667          1          1           2   
3         10.0        19.0    0.000000          1          1           2   
4         13.0        20.0    0.100000          3          1           4   

   freq_q1-q2  
0           0  
1           3  
2           0  
3           0  
4           2

In [36]:
df3_q1.head()

0           1           2           3          4          5   \
0   78.682992   87.635912   77.898819  -61.473692  44.053226  18.525178   
1   99.993008   55.174564   -2.049167   36.677249  85.412371 -45.989080   
2   62.709638   72.489519   10.889310  -45.772860  71.261772 -34.385969   
3   35.006791  -40.413219   53.450493  -45.069038  37.137247 -21.992808   
4  135.425154  187.445625  143.612776 -111.735024  56.977977 -70.101866   

          6           7           8           9     ...             86  \
0 -28.609312   47.452460  -86.095610   58.907952    ...       7.157678   
1  31.112590   76.453094  -74.456509  110.348369    ...     -64.475704   
2 -26.228285   18.224490 -113.496336  115.968702    ...      77.008272   
3 -28.184323  131.916699   41.891510   27.243861    ...     -36.975472   
4 -47.585533   59.575895  -56.992457  253.326808    ...     -34.521699   

          87         88         89         90         91          92  \
0  86.842601  38.238606 -25.909486   3.169638 -54.031532 -112.663659   
1  27.344039 -22.471263 -23.111044 -97.185489  13.815928  -24.577477   
2   5.414788 -26.222928  35.709896 -49.750098 -74.032807 -130.011004   
3  25.987250 -74.511655 -45.798322  42.739461 -17.318146   37.957786   
4  74.533560  -3.963831 -77.077944  27.673524 -87.661703 -146.777092   

          93          94         95  
0   1.619508   81.309565 -17.361949  
1  72.654378   58.654857 -19.836278  
2 -84.557644   10.153947 -30.314630  
3 -47.867102 -101.418604  -3.919247  
4   1.730535    5.950078 -12.494797  

[5 rows x 96 columns]

In [37]:
df3_q2.head()

0          1           2          3          4          5   \
0   69.938774  62.048677   65.471186 -65.080995  37.007811  11.953720   
1  100.523184  60.707327  114.135677  -2.311834  28.477254  -0.705671   
2   28.122773  -0.262905   -8.952834 -14.151883  18.908912  40.142980   
3   22.605501 -22.139145   18.226516 -20.430915  22.291252  35.642941   
4    1.158371  29.232888   66.428352 -43.949273  12.497666   6.992168   

           6           7          8           9     ...             86  \
0  -44.908971   39.077935 -71.113359   64.970021    ...      15.591466   
1  113.688085  117.816506 -93.662698   81.227491    ...     -84.838611   
2   62.036967   60.638759 -79.254361  108.044290    ...      27.926253   
3  -31.471534   61.818255 -40.677339   33.871990    ...      -7.491278   
4  -57.236839   32.344526 -56.251407   48.231540    ...      44.084751   

          87         88         89         90         91          92  \
0  96.113849  31.001480 -44.172052  21.397091 -46.908810 -100.162247   
1  54.099417 -65.543286 -23.634332 -85.303453  33.143694  -65.774337   
2  33.117176  66.706154  50.397498  42.266215 -29.920099  -53.378103   
3  44.788221   6.437781  39.981581 -43.237003  39.471643   -0.322382   
4  37.174410   0.356843 -29.468567  33.367771 -12.860933  -44.425541   

          93         94         95  
0  -1.978221  38.112752 -11.592531  
1  28.145018 -22.243213   0.169092  
2 -31.755547  77.061497   9.438696  
3  -6.074735  -4.417220   5.627428  
4   3.958389 -13.291972  -9.029029  

[5 rows x 96 columns]

In [38]:
print("Number of features in nlp dataframe :", df1.shape[1])
print("Number of features in preprocessed dataframe :", df2.shape[1])
print("Number of features in question1 w2v  dataframe :", df3_q1.shape[1])
print("Number of features in question2 w2v  dataframe :", df3_q2.shape[1])
print("Number of features in final dataframe  :", df1.shape[1]+df2.shape[1]+df3_q1.shape[1]+df3_q2.shape[1])

Number of features in nlp dataframe : 30
Number of features in preprocessed dataframe : 14
Number of features in question1 w2v  dataframe : 96
Number of features in question2 w2v  dataframe : 96
Number of features in final dataframe  : 236


In [ ]:
# storing the final features to csv file
if not os.path.isfile('final_features.csv'):
    df3_q1['id']=df1['id']
    df3_q2['id']=df1['id']
    df1  = df1.merge(df2, on='id',how='left')
    df2  = df3_q1.merge(df3_q2, on='id',how='left')
    result  = df1.merge(df2, on='id',how='left')
    result.to_csv('final_features.csv')

In [1]:
import pandas as pd
df=pd.read_csv("final_features.csv")

In [4]:
df.shape

(404290, 236)

In [3]:
df.head()

Unnamed: 0  id  is_duplicate  freq_quid1_x  freq_quid2_x  q1len_x  q2len_x  \
0           0   0             0             1             1       66       57   
1           1   1             0             4             1       51       88   
2           2   2             0             1             1       73       59   
3           3   3             0             1             1       50       65   
4           4   4             0             3             1       76       39   

   q1_n_words_x  q2_n_words_x  word_common_x    ...           86_y       87_y  \
0            14            12           12.0    ...      15.591466  96.113849   
1             8            13            8.0    ...     -84.838611  54.099417   
2            14            10           14.0    ...      27.926253  33.117176   
3            11             9           10.0    ...      -7.491278  44.788221   
4            13             7           13.0    ...      44.084751  37.174410   

        88_y       89_y       90_y       91_y        92_y       93_y  \
0  31.001480 -44.172052  21.397091 -46.908810 -100.162247  -1.978221   
1 -65.543286 -23.634332 -85.303453  33.143694  -65.774337  28.145018   
2  66.706154  50.397498  42.266215 -29.920099  -53.378103 -31.755547   
3   6.437781  39.981581 -43.237003  39.471643   -0.322382  -6.074735   
4   0.356843 -29.468567  33.367771 -12.860933  -44.425541   3.958389   

        94_y       95_y  
0  38.112752 -11.592531  
1 -22.243213   0.169092  
2  77.061497   9.438696  
3  -4.417220   5.627428  
4 -13.291972  -9.029029  

[5 rows x 236 columns]